In [58]:
import sqlite3
import pandas as pd
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

import warnings
# Not something I would include in production
# Included for prototyping / sake of time
warnings.filterwarnings("ignore")

<div>
<img src="erd.png" width="600"/>
</div>

# Simulate Driver's Seat app data

* Create sqlite database
* Use Shapely (Python) library to simulate geometry datatypes

### Create sqlite database

In [59]:
conn = sqlite3.connect('app') 
c = conn.cursor()

In [60]:
c.execute('''
          CREATE TABLE IF NOT EXISTS job
          (
          [job_id] INTEGER PRIMARY KEY, 
          [driver_id] INTEGER, 
          [employer_id] INTEGER, 
          [pickup_time_utc] TIMESTAMP, 
          [dropoff_time_utc] TIMESTAMP,
          [pickup_day] INTEGER, 
          [pickup_hour] INTEGER,
          [earnings_total] FLOAT
          )
          ''')

c.execute('''
          CREATE TABLE IF NOT EXISTS employer
          (
          [employer_id] INTEGER PRIMARY KEY, 
          [employer_name] STRING
          )
          ''')


c.execute('''
          CREATE TABLE IF NOT EXISTS driver
          (
          [driver_id] INTEGER PRIMARY KEY, 
          [driver_name] STRING,
          [timezone] STRING
          )
          ''')

c.execute('''
          CREATE TABLE IF NOT EXISTS driver_device_location
          (
          [location_id] INTEGER PRIMARY KEY, 
          [driver_id] INTTEGER, 
          [capture_time_utc] TIMESTAMP,
          [point] GEOMETRY
          )
          ''')

c.execute('''
          CREATE TABLE IF NOT EXISTS metro_area
          (
          [area_id] INTEGER PRIMARY KEY, 
          [area_name] STRING, 
          [shape] GEOMETRY
          )
          ''')

conn.commit()

In [61]:
c.execute('''
          INSERT INTO job (job_id, driver_id, employer_id, pickup_time_utc, dropoff_time_utc, 
          pickup_day, pickup_hour, earnings_total)

                VALUES
                (100, 1, 1, '2016-06-24 01:10:00.000', '2016-06-24 01:21:00.000', 1, 1, 21.92),
                (101, 2, 2, '2016-06-25 02:10:00.000', '2016-06-25 02:17:00.000', 2, 2, 12.30),
                (102 ,3, 2, '2016-06-26 03:10:00.000', '2016-06-26 03:13:00.000', 3, 3, 13.01),
                (104, 4, 2, '2016-06-26 02:10:00.000', '2016-06-26 02:13:00.000', 3, 2, 18.30),
                (105, 5, 1, '2016-06-22 19:10:00.000', '2016-06-22 19:10:00.000', 2, 1, 6.20),
                (106, 5, 1, '2016-06-22 19:10:00.000', '2016-06-22 19:10:00.000', 3, 1, 8.20)
          ''')

c.execute('''
          INSERT INTO employer (employer_id, employer_name)

                VALUES
                (1, 'Uber'),
                (2, 'Doordash')
          ''')

c.execute('''
          INSERT INTO driver (driver_id, driver_name, timezone)

                VALUES
                (1, 'Jake', 'PST'),
                (2, 'Robin', 'EST'),
                (3, 'Pablo', 'MST'),
                (4, 'Casey', 'EST'),
                (5, 'Ryan', 'CST'),
                (6, 'Sarah', 'PST')
          ''')

c.execute('''
          INSERT INTO driver_device_location (location_id, driver_id, capture_time_utc, point)

                VALUES
                (1001, 1, '2016-06-24 01:10:00', '0.5,0.5'),
                (1002, 1, '2016-06-24 01:16:00', '0.5,0.6'),
                (1003, 1, '2016-06-24 01:21:00', '0.5,0.7'),
                (1004, 2, '2016-06-25 02:10:00', '0.9,0.9'),
                (1005, 2, '2016-06-25 02:12:00', '0.95,0.9'),
                (1006, 2, '2016-06-25 02:14:00', '1.01,0.95'),
                (1007, 2, '2016-06-25 02:16:00', '1.05,0.99'),
                (1008, 2, '2016-06-25 02:17:00', '1.10,0.99'),
                (1009, 3, '2016-06-26 03:12:00', '0.05,0.06'),
                (1010, 4, '2016-06-26 02:12:00', '0.05,0.06')
                
          ''')

c.execute('''
          INSERT INTO metro_area (area_id, area_name, shape)

                VALUES
                (22, 'Los Angeles-Long Beach-Glendale MD', '(0,0) (0,1) (1,1) (1,0)'),
                (23, 'Anaheim-Santa Ana-Irvine MD', '(1,0) (1,1) (2,1) (2,0)')
          ''')

conn.commit()

### Use Shapely (Python) library to simulate geometry datatypes

In [62]:
# Join jobs to driver_device_location so that we can get consistent ride locations 
# (pick up an drop off locations are often missing)

# Join job to driver_device_location based on:
# 1) Matching driver_id 
# 2) Driver device location time is between job pickup time and dropoff time

c.execute('''
          SELECT job_id, point
          FROM job
          JOIN driver on job.driver_id = driver.driver_id
          JOIN driver_device_location on job.driver_id = driver_device_location.driver_id
              AND driver_device_location.capture_time_utc BETWEEN job. pickup_time_utc AND job.dropoff_time_utc 
          ''')

trips = pd.DataFrame(c.fetchall() , columns=['job_id', 'point'])

In [63]:
c.execute('''
          SELECT area_id, area_name, shape
          FROM metro_area
          ''')

metro_areas = pd.DataFrame(c.fetchall() , columns=['area_id', 'area_name', 'shape'])

In [64]:
# Converts string to geometry Point datatype
def create_point(coord):
    ps = coord.split(",")
    return Point(float(ps[0]), float(ps[1]))

# Converts string to geometry Polygon datatype
def create_polygon(coords):
    ps = coords.replace('(','').replace(')','').split(" ")
    points = []
    for i in ps:
        points.append(create_point(i))
    return Polygon(points)

In [65]:
# Convert 'shape' column in metro_areas to Polygon datatype
for ind in metro_areas.index:
    metro_areas["shape"][ind] = create_polygon(metro_areas['shape'][ind])
    
# Convert 'point' column in metro_areas to Point datatype
for ind in trips.index:
    trips['point'][ind] = create_point(trips['point'][ind])

# Power Average Hourly Pay View
1. Lookup and store metro area_name for each driver_device_location Point
2. Determine dominant metro_area for each job
3. Filter jobs to specific metro area
4. Calculate total earnings per job
5. Determine day of week and hour bins
6. Calculate average pay grouped by hour of day and day of week

Assumptions:
* Choosing to use location data from driver_device_location table vs. location data from job table for consistency
* Ignoring timezones for simplicity
* Calculations for all rideshare, all delivery. Employer data ignored.
* Assume there is already an earnings_total column in job table which is sum of earnings_pay, earnings_tip, and earnings_incentive

### 1. Lookup and store metro area_name for each driver_device_location Point

In [66]:
# Create new column to house metro area_name in trips
trips["metro_area"] = np.nan
# For each driver_device_location Point
for ind in trips.index:
    point = trips['point'][ind]

    # For each metro_area
    for ind2 in metro_areas.index:
        polygon = metro_areas['shape'][ind2]
        
        # Determine which metro_area Polygon contains Point
        if polygon.contains(point):
            
            # Store metro area in trips
            trips["metro_area"][ind] = metro_areas['area_name'][ind2]
            
trips

,job_id,point,metro_area
0,100,POINT (0.5 0.6),Los Angeles-Long Beach-Glendale MD
1,100,POINT (0.5 0.7),Los Angeles-Long Beach-Glendale MD
2,101,POINT (0.95 0.9),Los Angeles-Long Beach-Glendale MD
3,101,POINT (1.01 0.95),Anaheim-Santa Ana-Irvine MD
4,101,POINT (1.05 0.99),Anaheim-Santa Ana-Irvine MD
5,101,POINT (1.1 0.99),Anaheim-Santa Ana-Irvine MD
6,102,POINT (0.05 0.06),Los Angeles-Long Beach-Glendale MD
7,104,POINT (0.05 0.06),Los Angeles-Long Beach-Glendale MD


### 2. Determine dominant metro_area for each job
"Dominant metro_area" defined as the metro_area that houses the majority of points from driver_device_location
- Take first metro_area in the case of a tie

In [67]:
# Determine which metro_area is dominant for each job

# Create new column to house dominant metro area_name in trips
trips["dominant_metro_area"] = np.nan

for j in trips["job_id"].unique():
    job_trips = trips.loc[trips['job_id'] == j]
    ma_mode = np.array(job_trips["metro_area"].mode())
    
    if len(ma_mode) > 1: # Edge case where there are equal trip Points in multiple metro areas
        trips["dominant_metro_area"] = ma_mode[0][0]
    else:
        trips["dominant_metro_area"] = np.where(trips['job_id'] == j, ma_mode[0], trips["dominant_metro_area"])

In [68]:
job_area = trips.drop(['point', 'metro_area'], axis=1)
job_area = job_area.drop_duplicates()
job_area

,job_id,dominant_metro_area
0,100,Los Angeles-Long Beach-Glendale MD
2,101,Anaheim-Santa Ana-Irvine MD
6,102,Los Angeles-Long Beach-Glendale MD
7,104,Los Angeles-Long Beach-Glendale MD


### 3. Filter jobs to specific metro area

In [74]:
job_area_la = job_area.loc[job_area['dominant_metro_area'] == "Los Angeles-Long Beach-Glendale MD"]
job_area_la

,job_id,dominant_metro_area
0,100,Los Angeles-Long Beach-Glendale MD
6,102,Los Angeles-Long Beach-Glendale MD
7,104,Los Angeles-Long Beach-Glendale MD


### 4. Calculate total pay per job
In production this would be sum of earnings pay, tip, and incentive attributes from the jobs table

### 5. Determine day of week and hour bins
- I'm simply taking the pickup time hour and pickup day of week
- In practice, I'd would consider taking the parts of the day of week and hour
    - eg. for trip spanning 3:40pm to 4:30pm, 2/5 of the total earnings would count towards 3pm and 3/5 of the earnings would count towards 3pm
- In practice, I would use PostgreSQL timestamptz datatype to handle time zones
- I ended up adding "pickup_day" and "pickup_hour" columns directly to the db. Normally I would use SQL to extract these from the timestamp stored in the db but dates in SQLite work differently than in PostgreSQL

In [78]:
# Retrieve total earnings and job time
c.execute('''
          SELECT job_id, pickup_time_utc, earnings_total, pickup_hour,
              case 'pickup_day'
                  when 0 then 'Sunday'
                  when 1 then 'Monday'
                  when 2 then 'Tuesday'
                  when 3 then 'Wednesday'
                  when 4 then 'Thursday'
                  when 5 then 'Friday'
                  else 'Saturday' 
                end as day_of_week
          FROM job
          JOIN driver on job.driver_id = driver.driver_id 
          ''')

job_earnings = pd.DataFrame(c.fetchall() , columns=['job_id', 'pickup_time_utc', 'earnings_total',
                                                    'pickup_hour', 'pickup_day'])

job_earnings

,job_id,pickup_time_utc,earnings_total,pickup_hour,pickup_day
0,100,2016-06-24 01:10:00.000,21.92,1,Saturday
1,101,2016-06-25 02:10:00.000,12.30,2,Saturday
2,102,2016-06-26 03:10:00.000,13.01,3,Saturday
3,104,2016-06-26 02:10:00.000,18.30,2,Saturday
4,105,2016-06-22 19:10:00.000,6.20,1,Saturday
5,106,2016-06-22 19:10:00.000,8.20,1,Saturday


### 6. Calculate average pay grouped by hour of day and day of week
- Assumption: One trip per hour. 
    - In production, we'd have to add the earnings from all trips (or parts of trips) that fall within a given hour

In [79]:
trips_la = pd.merge(job_area_la, job_earnings)
trips_la

,job_id,dominant_metro_area,pickup_time_utc,earnings_total,pickup_hour,pickup_day
0,100,Los Angeles-Long Beach-Glendale MD,2016-06-24 01:10:00.000,21.92,1,Saturday
1,102,Los Angeles-Long Beach-Glendale MD,2016-06-26 03:10:00.000,13.01,3,Saturday
2,104,Los Angeles-Long Beach-Glendale MD,2016-06-26 02:10:00.000,18.30,2,Saturday


In [77]:
# Sum earnings_total, grouped by pickup_hour and pickup_day

<div>
<img src="view.png" width="300"/>
</div>